# Evaluating Opik's Hallucination Metric

*This cookbook was created from a Jypyter notebook which can be found [here](TBD).*

For this guide we will be evaluating the Hallucination metric included in the LLM Evaluation SDK which will showcase both how to use the `evaluation` functionality in the platform as well as the quality of the Hallucination metric included in the SDK.

In [ ]:
%pip install pyarrow fsspec huggingface_hub --quiet

In [1]:
# Configure OpenAI
import os
import getpass

os.environ["COMET_URL_OVERRIDE"] = "http://localhost:5173/api"
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key: ")

We will be using the [HaluBench dataset](https://huggingface.co/datasets/PatronusAI/HaluBench?library=pandas) which according to this [paper](https://arxiv.org/pdf/2407.08488) GPT-4o detects 87.9% of hallucinations. The first step will be to create a dataset in the platform so we can keep track of the results of the evaluation.

In [2]:
# Create dataset
from opik import Opik, DatasetItem
import pandas as pd

client = Opik()
try:
    # Create dataset
    dataset = client.create_dataset(name="HaluBench", description="HaluBench dataset")

    # Insert items into dataset
    df = pd.read_parquet("hf://datasets/PatronusAI/HaluBench/data/test-00000-of-00001.parquet")
    df = df.sample(n=500, random_state=42)

    dataset_records = [
        DatasetItem(
            input = {
                "input": x["question"],
                "context": [x["passage"]],
                "output": x["answer"]
            },
            expected_output = {"expected_output": x["label"]}
        )
        for x in df.to_dict(orient="records")
    ]
    
    dataset.insert(dataset_records)

except Exception as e:
    print(e)

status_code: 409, body: {'errors': ['Dataset already exists']}


In [4]:
from opik.evaluation.metrics import Hallucination
from opik.evaluation import evaluate
from opik.evaluation.metrics import base_metric, score_result
from opik import Opik, DatasetItem
import pandas as pd

client = Opik()

class CheckHallucinated(base_metric.BaseMetric):
    def __init__(self, name: str):
        self.name = name

    def score(self, hallucination_score, expected_hallucination_score, **kwargs):
        expected_hallucination_score = 1 if expected_hallucination_score == "FAIL" else 0
        
        return score_result.ScoreResult(
            value= None if hallucination_score is None else hallucination_score == expected_hallucination_score,
            name=self.name,
            reason=f"Got the hallucination score of {hallucination_score} and expected {expected_hallucination_score}",
            scoring_failed=hallucination_score is None
        )

def evaluation_task(x: DatasetItem):
    metric = Hallucination()
    try:
        metric_score = metric.score(
            input= x.input["input"],
            context= x.input["context"],
            output= x.input["output"]
        )
        hallucination_score = metric_score.value
        hallucination_reason = metric_score.reason
    except Exception as e:
        print(e)
        hallucination_score = None
        hallucination_reason = str(e)
    
    return {
        "hallucination_score": hallucination_score,
        "hallucination_reason": hallucination_reason,
        "expected_hallucination_score": x.expected_output["expected_output"]
    }

dataset = client.get_dataset(name="HaluBench")

res = evaluate(
    experiment_name="Check Comet Metric",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[CheckHallucinated(name="Detected hallucination")]
)

Scoring outputs: 100%|██████████| 500/500 [00:00<00:00, 513253.06it/s]


╭─ HaluBench (500 samples) ────────────╮
│                                      │
│ Total time:        00:00:53          │
│ Number of samples: 500               │
│                                      │
│ Detected hallucination: 0.8020 (avg) │
│                                      │
╰──────────────────────────────────────╯

Uploading results to Opik ...

We can see that the hallucination metric is able to detect ~80% of the hallucinations contained in the dataset.